## Prepare Notebook

In [1]:
import arviz as az
from datetime import datetime
from itertools import chain
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np
import numpy.typing as npt
import pandas as pd
import pymc as pm
import pymc.sampling_jax
import seaborn as sns
from scipy.special import expit


plt.style.use("bmh")
plt.rcParams["figure.figsize"] = [12, 7]
plt.rcParams["figure.dpi"] = 100
plt.rcParams["figure.facecolor"] = "white"

%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = "retina"

/Users/juanitorduz/.local/share/virtualenvs/website_projects-1IZj_WTw/lib/python3.10/site-packages/pymc/sampling/jax.py:37: UserWarning: This module is experimental.
  warnings.warn("This module is experimental.")


## Generate Data

In [2]:
seed: int = sum(map(ord, "retention"))
rng: np.random.Generator = np.random.default_rng(seed=seed)


class CohortDataGenerator:
    def __init__(
        self,
        rng: np.random.Generator,
        start_cohort: str,
        n_cohorts,
        user_base: int = 10_000,
    ) -> None:
        self.rng = rng
        self.start_cohort = start_cohort
        self.n_cohorts = n_cohorts
        self.user_base = user_base

    def _generate_cohort_labels(self) -> pd.DatetimeIndex:
        return pd.period_range(
            start="2020-01-01", periods=self.n_cohorts, freq="M"
        ).to_timestamp()

    def _generate_cohort_sizes(self) -> npt.NDArray[np.int_]:
        ones = np.ones(shape=self.n_cohorts)
        trend = ones.cumsum() / ones.sum()
        return (
            (self.user_base * trend * rng.gamma(shape=1, scale=1, size=self.n_cohorts))
            .round()
            .astype(int)
        )

    def _generate_dataset_base(self) -> pd.DataFrame:
        cohorts = self._generate_cohort_labels()
        n_users = self._generate_cohort_sizes()
        data_df = pd.merge(
            left=pd.DataFrame(data={"cohort": cohorts, "n_users": n_users}),
            right=pd.DataFrame(data={"period": cohorts}),
            how="cross",
        )
        data_df["age"] = (data_df["period"].max() - data_df["cohort"]).dt.days
        data_df["cohort_age"] = (data_df["period"] - data_df["cohort"]).dt.days
        data_df = data_df.query("cohort_age >= 0")
        return data_df

    def _generate_retention_rates(self, data_df: pd.DataFrame) -> pd.DataFrame:
        data_df["retention_true_mu"] = (
            -data_df["cohort_age"] / (data_df["age"] + 1)
            + 0.8 * np.sin(2 * np.pi * data_df["period"].dt.dayofyear / 365)
            + 0.5 * np.cos(2 * 3 * np.pi * data_df["period"].dt.dayofyear / 365)
            - 0.5 * np.log1p(data_df["age"])
            - 0.1
        )
        data_df["retention_true"] = expit(data_df["retention_true_mu"])
        return data_df

    def _generate_user_history(self, data_df: pd.DataFrame) -> pd.DataFrame:
        data_df["n_active_users"] = rng.binomial(
            n=data_df["n_users"], p=data_df["retention_true"]
        )
        return data_df

    def run(
        self,
    ) -> pd.DataFrame:
        data_df: pd.DataFrame = self._generate_dataset_base()
        return data_df.pipe(self._generate_retention_rates).pipe(
            self._generate_user_history
        )

In [3]:
ones = np.ones(shape=35)
trend = ones.cumsum() / ones.sum()

In [4]:
cohort_generator = CohortDataGenerator(rng=rng, start_cohort="2020-01-01", n_cohorts=35)
data_df = cohort_generator.run()

data_df.head()

,cohort,n_users,period,age,cohort_age,retention_true_mu,retention_true,n_active_users
0,2020-01-01,206,2020-01-01,1035,0,-3.058457,0.044854,13
1,2020-01-01,206,2020-02-01,1035,31,-3.223590,0.038288,9
2,2020-01-01,206,2020-03-01,1035,60,-3.435493,0.031204,10
3,2020-01-01,206,2020-04-01,1035,91,-2.840105,0.055195,7
4,2020-01-01,206,2020-05-01,1035,121,-2.497917,0.076004,19
